# Problema

Predecir el coste del seguro

# El set de datos

* age: age of primary beneficiary

* sex: insurance contractor gender, female, male

* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

* children: Number of children covered by health insurance / Number of dependents

* smoker: Smoking

* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

* charges: Individual medical costs billed by health insurance



In [5]:
# imports
import pandas as pd
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [6]:
ruta = "/content/insurance.csv"
data = pd.read_csv(ruta)

In [4]:
print(data.shape)
data.head()

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Objetivo

Generar un model de regresión capaz de predecir el valor del seguro en base a las características del cliente.

* Aplicar las técnicas oportunas de procesamiento de datos

* Valorar diferentes modelos de regresión

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

## Implementación

In [7]:
## LIMPIEZA DE DATOS

import pandas as pd

# Lee el archivo CSV
df = pd.read_csv('/content/insurance.csv')

# Encuentra las celdas vacías en el DataFrame
celdas_vacias = df.isnull().values.any()

if celdas_vacias:
    print("Se encontraron celdas vacías en el archivo CSV.")
else:
    print("No se encontraron celdas vacías en el archivo CSV.")

# Encuentra filas duplicadas en el DataFrame
filas_duplicadas = df[df.duplicated()]

if not filas_duplicadas.empty:
    print("Se encontraron filas duplicadas en el archivo CSV:")
    print(filas_duplicadas)
else:
    print("No se encontraron filas duplicadas en el archivo CSV.")

df_sin_duplicados = df.drop_duplicates()

# Guarda el DataFrame resultante en un nuevo archivo CSV si deseas
df_sin_duplicados.to_csv('archivo_sin_duplicados.csv', index=False)

# Imprime un mensaje indicando cuántas filas se eliminaron, si es necesario
filas_eliminadas = len(df) - len(df_sin_duplicados)
if filas_eliminadas > 0:
    print(f"Se eliminaron {filas_eliminadas} filas duplicadas.")
else:
    print("No se encontraron filas duplicadas en el archivo CSV.")


No se encontraron celdas vacías en el archivo CSV.
Se encontraron filas duplicadas en el archivo CSV:
     age   sex    bmi  children smoker     region    charges
581   19  male  30.59         0     no  northwest  1639.5631
Se eliminaron 1 filas duplicadas.


In [8]:
## NORMALIZACIÓN

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Cargar datos desde el archivo CSV
df = pd.read_csv('/content/archivo_sin_duplicados.csv')

# Inicializar el objeto MinMaxScaler
scaler = MinMaxScaler()

# Normalizar las variables numéricas (age y bmi)
df[['age', 'bmi']] = scaler.fit_transform(df[['age', 'bmi']])

# Opcional: Guardar el DataFrame con la columna bmi normalizada en un nuevo archivo CSV
df.to_csv('datos_con_bmi_y_age_normalizado.csv', index=False)

print(df.head())


        age     sex       bmi  children smoker     region      charges
0  0.021739  female  0.321227         0    yes  southwest  16884.92400
1  0.000000    male  0.479150         1     no  southeast   1725.55230
2  0.217391    male  0.458434         3     no  southeast   4449.46200
3  0.326087    male  0.181464         0     no  northwest  21984.47061
4  0.304348    male  0.347592         0     no  northwest   3866.85520


In [9]:
df['region'] = df['region'].map({'northeast': 0, 'northwest': 1, 'southeast': 2, 'southwest': 3})
df['sex'] = df['sex'].map({'female': 0, 'male': 1})
df['smoker'] = df['smoker'].map({'yes': 0, 'no': 1})

print(df.head())


        age  sex       bmi  children  smoker  region      charges
0  0.021739    0  0.321227         0       0       3  16884.92400
1  0.000000    1  0.479150         1       1       2   1725.55230
2  0.217391    1  0.458434         3       1       2   4449.46200
3  0.326087    1  0.181464         0       1       1  21984.47061
4  0.304348    1  0.347592         0       1       1   3866.85520


In [10]:
from sklearn.model_selection import train_test_split

# Separar las características (X) de la variable objetivo (y)
X = df.drop('charges', axis=1)
y = df['charges']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir las dimensiones de los conjuntos de entrenamiento y prueba
print("Dimensiones del conjunto de entrenamiento:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nDimensiones del conjunto de prueba:")
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

Dimensiones del conjunto de entrenamiento:
X_train: (1069, 6)
y_train: (1069,)

Dimensiones del conjunto de prueba:
X_test: (268, 6)
y_test: (268,)


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


## MODELO DE REGRESIÓN LINEAL

# Codificar variables categóricas utilizando one-hot encoding
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

# Inicializar el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento codificados
model.fit(X_train_encoded, y_train)

# Predecir los valores del conjunto de entrenamiento y prueba
y_train_pred = model.predict(X_train_encoded)
y_test_pred = model.predict(X_test_encoded)

# Calcular las métricas de evaluación
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Imprimir las métricas de evaluación
print("Métricas de evaluación del modelo de regresión lineal:")
print("Conjunto de entrenamiento:")
print("  MSE:", train_mse)
print("  R^2:", train_r2)
print("Conjunto de prueba:")
print("  MSE:", test_mse)
print("  R^2:", test_r2)


Métricas de evaluación del modelo de regresión lineal:
Conjunto de entrenamiento:
  MSE: 37005531.72811555
  R^2: 0.7297182858804965
Conjunto de prueba:
  MSE: 35493102.61165049
  R^2: 0.8068466322629113


**- Conjunto de entrenamiento:**
*MSE (Error Cuadrático Medio):* 37,005,531.73
Esto significa que, en promedio, las predicciones del modelo tienen un error cuadrático de aproximadamente 37,005,531.73 unidades al cuadrado respecto a los valores reales en el conjunto de entrenamiento.
Un MSE más bajo indica un mejor ajuste del modelo a los datos.
*R^2 (Coeficiente de Determinación):* 0.73
Esto significa que aproximadamente el 73% de la variabilidad en la variable dependiente puede ser explicada por el modelo de regresión lineal en el conjunto de entrenamiento.
Un valor de R^2 más cercano a 1 indica un mejor ajuste del modelo a los datos.

**- Conjunto de prueba:**
*MSE (Error Cuadrático Medio):* 35,493,102.61
Similar al conjunto de entrenamiento, pero aplicado al conjunto de prueba. El MSE aquí es ligeramente menor que en el conjunto de entrenamiento, lo cual es positivo.
*R^2 (Coeficiente de Determinación):* 0.81
Esto significa que aproximadamente el 81% de la variabilidad en la variable dependiente puede ser explicada por el modelo de regresión lineal en el conjunto de prueba.

El valor de R^2 en el conjunto de prueba es más alto que en el conjunto de entrenamiento, lo cual sugiere que el modelo generaliza bien a datos nuevos y no está sobreajustando demasiado los datos de entrenamiento.
En resumen, tus resultados muestran que el modelo de regresión lineal parece estar capturando una cantidad significativa de la variabilidad en los datos y generalizando bien a nuevos datos en el conjunto de prueba.


In [15]:
## VALIDACIÓN CRUZADA

from sklearn.model_selection import cross_val_score

# Inicializar un nuevo modelo de regresión lineal
model = LinearRegression()

# Realizar validación cruzada con 5 folds
cv_scores = cross_val_score(model, X_train_encoded, y_train, cv=5, scoring='neg_mean_squared_error')

# Convertir los scores a MSE positivos
cv_scores = -cv_scores

# Calcular el promedio y la desviación estándar de los scores
cv_mse_mean = cv_scores.mean()
cv_mse_std = cv_scores.std()

# Imprimir los resultados de la validación cruzada
print("Resultados de validación cruzada del modelo de regresión lineal:")
print("  MSE promedio:", cv_mse_mean)
print("  Desviación estándar del MSE:", cv_mse_std)


Resultados de validación cruzada del modelo de regresión lineal:
  MSE promedio: 37523328.96701016
  Desviación estándar del MSE: 4919524.258794246


 Estos resultados sugieren que el modelo tiene un rendimiento consistente en la predicción de nuevos datos, pero puede haber cierta variabilidad en su rendimiento en diferentes conjuntos de datos de validación cruzada.

In [16]:
## ÁRBOL DE DECISÓN

from sklearn.tree import DecisionTreeRegressor

# Inicializar el modelo de Árbol de Decisión para regresión
dt_model = DecisionTreeRegressor()

# Entrenar el modelo con los datos de entrenamiento codificados
dt_model.fit(X_train_encoded, y_train)

# Predecir los valores del conjunto de entrenamiento y prueba
y_train_pred_dt = dt_model.predict(X_train_encoded)
y_test_pred_dt = dt_model.predict(X_test_encoded)

# Calcular las métricas de evaluación
train_mse_dt = mean_squared_error(y_train, y_train_pred_dt)
test_mse_dt = mean_squared_error(y_test, y_test_pred_dt)
train_r2_dt = r2_score(y_train, y_train_pred_dt)
test_r2_dt = r2_score(y_test, y_test_pred_dt)

# Imprimir las métricas de evaluación
print("Métricas de evaluación del modelo de Árbol de Decisión para regresión:")
print("Conjunto de entrenamiento:")
print("  MSE:", train_mse_dt)
print("  R^2:", train_r2_dt)
print("Conjunto de prueba:")
print("  MSE:", test_mse_dt)
print("  R^2:", test_r2_dt)


Métricas de evaluación del modelo de Árbol de Decisión para regresión:
Conjunto de entrenamiento:
  MSE: 0.0
  R^2: 1.0
Conjunto de prueba:
  MSE: 43586100.709388606
  R^2: 0.762804558658592


**Error Cuadrático Medio (MSE):**
Para el conjunto de entrenamiento: MSE = 0.0
Para el conjunto de prueba: MSE = 42,918,508.42
El MSE es una medida de la calidad del ajuste del modelo a los datos. En el conjunto de entrenamiento, un MSE de 0.0 sugiere que el modelo se ajusta perfectamente a los datos de entrenamiento, lo cual puede ser una señal de overfitting. En el conjunto de prueba, el MSE es bastante alto, lo que indica que el modelo tiene dificultades para generalizar a datos no vistos.
**Coeficiente de Determinación (R^2):**
Para el conjunto de entrenamiento: R^2 = 1.0
Para el conjunto de prueba: R^2 = 0.7664375940740764
El coeficiente de determinación, o R^2, es una medida que indica qué tan bien se ajustan los valores observados a los valores predichos por el modelo. Un valor de R^2 igual a 1.0 en el conjunto de entrenamiento indica un ajuste perfecto del modelo a estos datos. Sin embargo, un R^2 de 0.77 en el conjunto de prueba sugiere que el modelo explica aproximadamente el 77% de la variabilidad en los datos de prueba.

En resumen, los resultados indican que el modelo de Árbol de Decisión se ajusta perfectamente a los datos de entrenamiento, pero tiene dificultades para generalizar a datos nuevos, como se observa en el conjunto de prueba. Esto sugiere un posible problema de overfitting, donde el modelo se adapta demasiado a los detalles específicos de los datos de entrenamiento y no logra capturar la verdadera relación subyacente entre las variables. Es posible que se necesiten técnicas de regularización o ajustes en los hiperparámetros del modelo para mejorar su capacidad de generalización.

In [17]:
## VALIDACIÓN CRUZADA

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

# Inicializar un nuevo modelo de Árbol de Decisión para regresión
tree_model = DecisionTreeRegressor()

# Realizar validación cruzada con 5 folds
cv_scores = cross_val_score(tree_model, X_train_encoded, y_train, cv=5, scoring='neg_mean_squared_error')

# Convertir los scores a MSE positivos
cv_scores = -cv_scores

# Calcular el promedio y la desviación estándar de los scores
cv_mse_mean = cv_scores.mean()
cv_mse_std = cv_scores.std()

# Imprimir los resultados de la validación cruzada
print("Resultados de validación cruzada del modelo de Árbol de Decisión para regresión:")
print("  MSE promedio:", cv_mse_mean)
print("  Desviación estándar del MSE:", cv_mse_std)


Resultados de validación cruzada del modelo de Árbol de Decisión para regresión:
  MSE promedio: 45134024.9410813
  Desviación estándar del MSE: 6870429.911294806


El modelo de Árbol de Decisión para regresión tiene un MSE promedio de aproximadamente 46,631,328.45 con una desviación estándar de aproximadamente 6,863,005.71. Estos resultados sugieren que el modelo tiene un rendimiento consistente en la predicción de nuevos datos, pero puede haber cierta variabilidad en su rendimiento en diferentes conjuntos de datos de validación cruzada.

In [18]:
## RANDOM FOREST

from sklearn.ensemble import RandomForestRegressor

# Inicializar el modelo de Bosque Aleatorio para regresión
rf_model = RandomForestRegressor()

# Entrenar el modelo con los datos de entrenamiento codificados
rf_model.fit(X_train_encoded, y_train)

# Predecir los valores del conjunto de entrenamiento y prueba
y_train_pred_rf = rf_model.predict(X_train_encoded)
y_test_pred_rf = rf_model.predict(X_test_encoded)

# Calcular las métricas de evaluación
train_mse_rf = mean_squared_error(y_train, y_train_pred_rf)
test_mse_rf = mean_squared_error(y_test, y_test_pred_rf)
train_r2_rf = r2_score(y_train, y_train_pred_rf)
test_r2_rf = r2_score(y_test, y_test_pred_rf)

# Imprimir las métricas de evaluación
print("Métricas de evaluación del modelo de Bosque Aleatorio para regresión:")
print("Conjunto de entrenamiento:")
print("  MSE:", train_mse_rf)
print("  R^2:", train_r2_rf)
print("Conjunto de prueba:")
print("  MSE:", test_mse_rf)
print("  R^2:", test_r2_rf)


Métricas de evaluación del modelo de Bosque Aleatorio para regresión:
Conjunto de entrenamiento:
  MSE: 3428437.760868329
  R^2: 0.9749593103656042
Conjunto de prueba:
  MSE: 22096877.82613619
  R^2: 0.8797488510572694


**- Error Cuadrático Medio (MSE):**
Para el conjunto de entrenamiento: MSE = 3,551,052.10
Para el conjunto de prueba: MSE = 21,366,706.53
El MSE es una medida de la calidad del ajuste del modelo a los datos. Un MSE más bajo indica un mejor ajuste del modelo a los datos. En este caso, el MSE es significativamente menor en el conjunto de entrenamiento que en el conjunto de prueba, lo cual puede sugerir un posible sobreajuste del modelo en los datos de entrenamiento.

**- Coeficiente de Determinación (R^2):**
Para el conjunto de entrenamiento: R^2 = 0.974
Para el conjunto de prueba: R^2 = 0.884
El coeficiente de determinación, o R^2, es una medida que indica qué tan bien se ajustan los valores observados a los valores predichos por el modelo. Un valor de R^2 más cercano a 1 indica un mejor ajuste del modelo a los datos. En este caso, tanto en el conjunto de entrenamiento como en el de prueba, el R^2 es alto, lo que sugiere que el modelo explica una gran proporción de la variabilidad en los datos.

Los resultados indican que el modelo de Bosque Aleatorio tiene un rendimiento bastante sólido en términos de ajuste a los datos. Sin embargo, es importante prestar atención al posible sobreajuste, ya que el MSE es considerablemente menor en el conjunto de entrenamiento que en el conjunto de prueba.

In [19]:
## GRADIENT BOOSTING

from sklearn.ensemble import GradientBoostingRegressor

# Inicializar el modelo de Regresión de Gradient Boosting
gb_model = GradientBoostingRegressor()

# Entrenar el modelo con los datos de entrenamiento codificados
gb_model.fit(X_train_encoded, y_train)

# Predecir los valores del conjunto de entrenamiento y prueba
y_train_pred_gb = gb_model.predict(X_train_encoded)
y_test_pred_gb = gb_model.predict(X_test_encoded)

# Calcular las métricas de evaluación
train_mse_gb = mean_squared_error(y_train, y_train_pred_gb)
test_mse_gb = mean_squared_error(y_test, y_test_pred_gb)
train_r2_gb = r2_score(y_train, y_train_pred_gb)
test_r2_gb = r2_score(y_test, y_test_pred_gb)

# Imprimir las métricas de evaluación
print("Métricas de evaluación del modelo de Regresión de Gradient Boosting:")
print("Conjunto de entrenamiento:")
print("  MSE:", train_mse_gb)
print("  R^2:", train_r2_gb)
print("Conjunto de prueba:")
print("  MSE:", test_mse_gb)
print("  R^2:", test_r2_gb)


Métricas de evaluación del modelo de Regresión de Gradient Boosting:
Conjunto de entrenamiento:
  MSE: 14626569.013276456
  R^2: 0.8931701840243523
Conjunto de prueba:
  MSE: 18162559.772530515
  R^2: 0.9011593991887624


**- Error Cuadrático Medio (MSE):**
Para el conjunto de entrenamiento: MSE = 14,626,569.01
Para el conjunto de prueba: MSE = 18,239,470.74
El MSE es una medida de la calidad del ajuste del modelo a los datos. Cuanto menor sea el valor de MSE, mejor será el ajuste del modelo a los datos. En este caso, el MSE es más bajo en el conjunto de entrenamiento que en el conjunto de prueba, lo que sugiere que el modelo está generalizando bien a nuevos datos, pero aún existe cierto error en las predicciones.
**- Coeficiente de Determinación (R^2):**
Para el conjunto de entrenamiento: R^2 = 0.893
Para el conjunto de prueba: R^2 = 0.901
El coeficiente de determinación, o R^2, es una medida que indica qué tan bien se ajustan los valores observados a los valores predichos por el modelo. Un valor de R^2 más cercano a 1 indica un mejor ajuste del modelo a los datos. En este caso, tanto en el conjunto de entrenamiento como en el de prueba, el R^2 es alto, lo que sugiere que el modelo explica una gran proporción de la variabilidad en los datos.

En resumen, los resultados indican que el modelo de Regresión de Gradient Boosting tiene un buen rendimiento en términos de ajuste a los datos. La diferencia entre el MSE del conjunto de entrenamiento y el de prueba no es muy grande, lo que sugiere que el modelo generaliza bien a nuevos datos. Además, el alto valor de R^2 en ambos conjuntos indica que el modelo explica una gran parte de la variabilidad en los datos. Esto sugiere que el modelo de Gradient Boosting es una opción prometedora para este problema de regresión.



CONCLUSIONES:

Basándome en las métricas obtenidas en cada uno de los modelos (Regresión Lineal, Árbol de Decisión, Bosque Aleatorio y Regresión de Gradient Boosting), algunas conclusiones globales sobre su rendimiento son:

1. **Regresión Lineal**:
   - El modelo de regresión lineal parece funcionar bien, con MSE y R^2 razonables tanto en el conjunto de entrenamiento como en el de prueba. Sin embargo, los valores de MSE son relativamente altos en comparación con los otros modelos, lo que sugiere que podría haber margen para mejorar la precisión de las predicciones.

2. **Árbol de Decisión**:
   - El modelo de Árbol de Decisión muestra un ajuste perfecto a los datos de entrenamiento, lo que indica un posible sobreajuste. Aunque tiene un rendimiento decente en el conjunto de prueba, el MSE es considerablemente alto, lo que sugiere que el modelo puede no estar generalizando bien a nuevos datos.

3. **Bosque Aleatorio**:
   - El modelo de Bosque Aleatorio muestra un rendimiento sólido tanto en el conjunto de entrenamiento como en el de prueba. Aunque hay una diferencia entre el MSE del conjunto de entrenamiento y el de prueba, el modelo generaliza bastante bien, como se indica por los altos valores de R^2.

4. **Regresión de Gradient Boosting**:
   - La Regresión de Gradient Boosting también muestra un buen rendimiento en ambos conjuntos de datos. Tiene MSE y R^2 similares al Bosque Aleatorio, lo que indica que ambos modelos están compitiendo por el mejor rendimiento. Sin embargo, la Regresión de Gradient Boosting parece tener un ligero mejor desempeño en el conjunto de prueba.

En general, si priorizamos la simplicidad del modelo y un ajuste aceptable, la Regresión Lineal podría ser suficiente. Si buscamos un mejor rendimiento, los modelos de Bosque Aleatorio y Regresión de Gradient Boosting parecen ser opciones sólidas. Es importante tener en cuenta el balance entre rendimiento y complejidad del modelo al seleccionar el más adecuado para su caso de uso específico. Además, siempre es recomendable ajustar los hiperparámetros de los modelos y realizar validación cruzada para obtener una evaluación más completa de su rendimiento.